# 2D Image to 3D Model

## Data Preprocessing



In [2]:
import numpy as np

def normalize_image(image):
    return np.array(image) / 255.0

from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

2024-09-06 01:42:08.717180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading Material Properties From json

In [ ]:
import json
from PIL import Image
import numpy as np

def load_preprocess_img(img_path):
    
    img = Image.open(img_path)
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

with open('material_properties.json', 'r') as f:
    material_properties = json.load(f)

def get_material_prop(img_path, img_to_mod_map, material_properties):
    model_path = img_to_mod_map.get(img_path, None)

    if model_path is None:
        raise ValueError(f"No model found for img: {img_path}")

    materials = material_properties.get(model_path, None)

    if materials is None:
        raise ValueError(f"No materials found for model: {model_path}")
    return materials

def normalize_materials(material):
    max_shine = 1000

    normalized_material = {
        'Kd': material.get('Kd', [1.0, 1.0, 1.0]),
        'Ks': material.get('Ks', [0.0, 0.0, 0.0]),
        'Ns': material.get('Ns', 96.078431) / max_shine,
        'd': material.get('d', 1.0)
    }

    return normalized_material

def preprocess_image_with_material(img_path, img_to_mod_map, material_properties):
    img = load_preprocess_img(img_path)

    materials = get_material_prop(img_path, img_to_mod_map, material_properties)

    normalized_materials = normalize_materials(materials)

    return img, normalized_materials